## DATA ENGINEERING PLATFORMS (MSCA 31012)

### File      :   Class Exercise - Session 10 - Sample Blockchain Implementation

### Desc     :  Blockchain Explained in 7 Python Functions
### Date      :   06/11/2019

References: 
https://www.kdnuggets.com/2018/04/blockchain-explained-7-python-functions.html


In [1]:
import json
import hashlib
import random

Blockchain is it is an encrypted database that is public. At the heart of the blockchain is the hashing function. Without encryption, the blockchain will be easily manipulated and transactions will be able to be fraudulently inserted.

In [30]:
def hash_function(k):
    """Hashes our transaction."""
    if type(k) is not str:
        k = json.dumps(k, sort_keys=True)

        return hashlib.sha256(str(random.getrandbits(256)).encode('utf-8')).hexdigest()

The ‘state’ is the record of ownership. For example, if Tom have 10 coins and he gives 1 to Medium, then the state will be  {‘transaction’: {‘Tom’: 9, ‘Medium’: 1}}

In [12]:
def update_state(transaction, state):
    state = state.copy()

    for key in transaction:
        if key in state.keys():
            state[key] += transaction[key]
        else:
            state[key] = transaction[key]

    return state

The important thing to note is that overdrafts cannot exist. If there are only 10 coins in existence, then I cannot give 11 coins to someone. The below function verifies that the transaction we attempt to make is indeed valid. Also, a transaction must balance. I cannot give 5 coins and have the recipient receive 4 coins, since that would allow the destruction and creation of coins.

In [13]:
def valid_transaction(transaction, state):
    """A valid transaction must sum to 0."""
    if sum(transaction.values()) is not 0:
        return False

    for key in transaction.keys():
        if key in state.keys():
            account_balance = state[key]
        else:
            account_balance = 0

        if account_balance + transaction[key] < 0:
            return False

    return True

Now, we can make our block. The information from the previous block is read, and used to link it to the new block. This, too, is central to the idea of blockchain. Seemingly valid transactions can be attempted to fraudulently be inserted into the blockchain, but decrypting all the previous blocks is computationally (nearly) impossible, which preserves the integrity of the blockchain.

In [14]:
def make_block(transactions, chain):
    """Make a block to go into the chain."""
    parent_hash = chain[-1]['hash']
    block_number = chain[-1]['contents']['block_number'] + 1

    block_contents = {
        'block_number': block_number,
        'parent_hash': parent_hash,
        'transaction_count': block_number + 1,
        'transaction': transactions
    }

    return {'hash': hash_function(block_contents), 'contents': block_contents}

Below is a small helper function to check the hash of the previous block

In [15]:
def check_block_hash(block):
    expected_hash = hash_function(block['contents'])

    if block['hash'] is not expected_hash:
        raise

    return

Once we have assembled everything together, its time to create our block. We will now update the blockchain.

In [16]:
def check_block_validity(block, parent, state):
    parent_number = parent['contents']['block_number']
    parent_hash = parent['hash']
    block_number = block['contents']['block_number']

    for transaction in block['contents']['transaction']:
        if valid_transaction(transaction, state):
            state = update_state(transaction, state)
        else:
            raise

    check_block_hash(block)  # Check hash integrity

    if block_number is not parent_number + 1:
        raise

    if block['contents']['parent_hash'] is not parent_hash:
        raise

    return state

Before we are finished, the chain must be verified

In [17]:
def check_chain(chain):
    """Check the chain is valid."""
    if type(chain) is str:
        try:
            chain = json.loads(chain)
            assert (type(chain) == list)
        except ValueError:
            # String passed in was not valid JSON
            return False
    elif type(chain) is not list:
        return False

    state = {}

    for transaction in chain[0]['contents']['transaction']:
        state = update_state(transaction, state)

    check_block_hash(chain[0])
    parent = chain[0]

    for block in chain[1:]:
        state = check_block_validity(block, parent, state)
        parent = block

    return state

Finally, need a transaction function, which hangs all of the above together

In [18]:
def add_transaction_to_chain(transaction, state, chain):
    if valid_transaction(transaction, state):
        state = update_state(transaction, state)
    else:
        raise Exception('Invalid transaction.')

    my_block = make_block(state, chain)
    chain.append(my_block)

    for transaction in chain:
        check_chain(transaction)

    return state, chain

So, now we have our 7 functions. How do we interact with it? Well, first we need to start our chain with a Genesis Block. This is the inception of our new coin (or stock inventory, etc). Let us start with an example where Tom, will start off with 10 coins.

In [19]:
genesis_block = {
    'hash': hash_function({
        'block_number': 0,
        'parent_hash': None,
        'transaction_count': 1,
        'transaction': [{'Tom': 10}]
    }),
    'contents': {
        'block_number': 0,
        'parent_hash': None,
        'transaction_count': 1,
        'transaction': [{'Tom': 10}]
    },
}

block_chain = [genesis_block]
chain_state = {'Tom': 10}

Now, look what happens when I give some coin to Medium:

In [20]:
chain_state, block_chain = add_transaction_to_chain(transaction={'Tom': -1, 'Medium': 1}, state=chain_state, chain=block_chain)

In [21]:
block_chain

[{'contents': {'block_number': 0,
   'parent_hash': None,
   'transaction': [{'Tom': 10}],
   'transaction_count': 1},
  'hash': '0797706e1b5b3b65711ad57619ce8905a61a4f87203dc6df10b7b4a0e697824e'},
 {'contents': {'block_number': 1,
   'parent_hash': '0797706e1b5b3b65711ad57619ce8905a61a4f87203dc6df10b7b4a0e697824e',
   'transaction': {'Medium': 1, 'Tom': 9},
   'transaction_count': 2},
  'hash': '043792420641e956d36643ededf926c4eef025c26314b59e651f2044362ade12'}]

In [22]:
chain_state, block_chain = add_transaction_to_chain(transaction={'Tom': -1, 'Medium': 1}, state=chain_state, chain=block_chain)

In [23]:
block_chain

[{'contents': {'block_number': 0,
   'parent_hash': None,
   'transaction': [{'Tom': 10}],
   'transaction_count': 1},
  'hash': '0797706e1b5b3b65711ad57619ce8905a61a4f87203dc6df10b7b4a0e697824e'},
 {'contents': {'block_number': 1,
   'parent_hash': '0797706e1b5b3b65711ad57619ce8905a61a4f87203dc6df10b7b4a0e697824e',
   'transaction': {'Medium': 1, 'Tom': 9},
   'transaction_count': 2},
  'hash': '043792420641e956d36643ededf926c4eef025c26314b59e651f2044362ade12'},
 {'contents': {'block_number': 2,
   'parent_hash': '043792420641e956d36643ededf926c4eef025c26314b59e651f2044362ade12',
   'transaction': {'Medium': 2, 'Tom': 8},
   'transaction_count': 3},
  'hash': 'f246041914ee108c0bd82b85a8c781d1f937f5b51f13af7c86d0368223227e26'}]

In [24]:
chain_state, block_chain = add_transaction_to_chain(transaction={'Tom': -2, 'Medium': 2}, state=chain_state, chain=block_chain)

In [25]:
block_chain

[{'contents': {'block_number': 0,
   'parent_hash': None,
   'transaction': [{'Tom': 10}],
   'transaction_count': 1},
  'hash': '0797706e1b5b3b65711ad57619ce8905a61a4f87203dc6df10b7b4a0e697824e'},
 {'contents': {'block_number': 1,
   'parent_hash': '0797706e1b5b3b65711ad57619ce8905a61a4f87203dc6df10b7b4a0e697824e',
   'transaction': {'Medium': 1, 'Tom': 9},
   'transaction_count': 2},
  'hash': '043792420641e956d36643ededf926c4eef025c26314b59e651f2044362ade12'},
 {'contents': {'block_number': 2,
   'parent_hash': '043792420641e956d36643ededf926c4eef025c26314b59e651f2044362ade12',
   'transaction': {'Medium': 2, 'Tom': 8},
   'transaction_count': 3},
  'hash': 'f246041914ee108c0bd82b85a8c781d1f937f5b51f13af7c86d0368223227e26'},
 {'contents': {'block_number': 3,
   'parent_hash': 'f246041914ee108c0bd82b85a8c781d1f937f5b51f13af7c86d0368223227e26',
   'transaction': {'Medium': 4, 'Tom': 6},
   'transaction_count': 4},
  'hash': 'dc8d15f7c013ce3bd197c072fc3a524d2c32b494685c97e1c57b9d581209

In [26]:
chain_state, block_chain = add_transaction_to_chain(transaction={'Tom': -1, 'High': 1}, state=chain_state, chain=block_chain)

In [27]:
block_chain

[{'contents': {'block_number': 0,
   'parent_hash': None,
   'transaction': [{'Tom': 10}],
   'transaction_count': 1},
  'hash': '0797706e1b5b3b65711ad57619ce8905a61a4f87203dc6df10b7b4a0e697824e'},
 {'contents': {'block_number': 1,
   'parent_hash': '0797706e1b5b3b65711ad57619ce8905a61a4f87203dc6df10b7b4a0e697824e',
   'transaction': {'Medium': 1, 'Tom': 9},
   'transaction_count': 2},
  'hash': '043792420641e956d36643ededf926c4eef025c26314b59e651f2044362ade12'},
 {'contents': {'block_number': 2,
   'parent_hash': '043792420641e956d36643ededf926c4eef025c26314b59e651f2044362ade12',
   'transaction': {'Medium': 2, 'Tom': 8},
   'transaction_count': 3},
  'hash': 'f246041914ee108c0bd82b85a8c781d1f937f5b51f13af7c86d0368223227e26'},
 {'contents': {'block_number': 3,
   'parent_hash': 'f246041914ee108c0bd82b85a8c781d1f937f5b51f13af7c86d0368223227e26',
   'transaction': {'Medium': 4, 'Tom': 6},
   'transaction_count': 4},
  'hash': 'dc8d15f7c013ce3bd197c072fc3a524d2c32b494685c97e1c57b9d581209